In [91]:
import nltk
nltk.download('stopwords')
nltk.download('perluniprops')
nltk.download('nonbreaking_prefixes')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.


True

In [92]:
from ourDb import events_collection, total_events_collection
import re
import nltk
import string
from nltk.tokenize.moses import MosesTokenizer

from gensim.models import Phrases
from nltk.corpus import wordnet as wn

In [104]:
class PreprocessText:
    def __init__(self, categorizedEvents):
        """categorizedEvents should be a list of dictionaries each corresponding to an event"""
        self.X = []
        self.Y = []
        tokenizer = MosesTokenizer() #tokenizers are basically an advanced split
        for e in categorizedEvents:
            text = e[u'name'] + " " + e[u'description']
            text = tokenizer.tokenize(text)
            text = self.preprocess(text)
            self.X.append(text)
            self.Y.append(e[u'category'])
        self.phraseMl = Phrases(self.X, min_count=3) #learn ml model for phrase
        self.X = list(self.phraseMl[self.X]) #use ml model for phrases
        
    def matchNotX(self, strg, search=re.compile(r'[^!#$%&()*+,-./:;<=>?@\\^_}|{~0123456789]').search):
        """make sure word has something than punctuation"""
        return bool(search(strg)) #make sure word has something other than punctuation

    def preprocess(self, text):
        """Remove all useless words and punct, make lowercase"""
        stoplist = set('for a of the and to in . / '.split())
        stoplist = set(nltk.corpus.stopwords.words('english')) | stoplist | set(string.punctuation)
        return [word.strip(string.punctuation).lower() for word in text if word not in stoplist and self.matchNotX(word)]    
        
    def topBigrams(self, texts, n, tri=False):
        """Apply this to see into the data for which phrases gensim is probably choosing to combine"""
        flatTexts = []
        for text in texts:
            for word in text:
                flatTexts.append(word)
        bigram_measures = nltk.collocations.BigramAssocMeasures()
        trigram_measures = nltk.collocations.TrigramAssocMeasures()
        topAnswers = []
        if tri:
            finder = nltk.collocations.TrigramCollocationFinder.from_words(flatTexts)
            finder.apply_freq_filter(7)
            return finder.nbest(trigram_measures.pmi, n)
        else:
            finder = nltk.collocations.BigramCollocationFinder.from_words(flatTexts)
            finder.apply_freq_filter(7)
            return finder.nbest(bigram_measures.pmi, n)

In [105]:
def gatherCategorizedEvents():
    allCategorizedEvents = []
    allEvents = total_events_collection.find({}, {"category": 1, "description": 1, "name": 1, "_id": 0})
    count = 0
    for e in allEvents:
        count += 1
        if 'category' in e and 'description' in e and 'name' in e:
            allCategorizedEvents.append(e)
    print count, "events and using the", len(allCategorizedEvents), "categorized events"
    return allCategorizedEvents[:5]

X = gatherCategorizedEvents()
p = PreprocessText(X)
p.X

497 events and using the 245 categorized events


[[u'one-day',
  u'meditation',
  u'workshop',
  u'free',
  u'event',
  u'this',
  u'one-day',
  u'workshop',
  u'ucla',
  u'covel',
  u'common',
  u'present',
  u'meditation',
  u'could',
  u'bring',
  u'us',
  u'emotion',
  u'wellness',
  u'lower',
  u'stress',
  u'presence',
  u'personal',
  u'interactions',
  u'healthier',
  u'approach',
  u'life',
  u'instructors',
  u'studied',
  u'largest',
  u'buddhist',
  u'academy',
  u'tibet',
  u'many',
  u'years',
  u'this',
  u'3-session-courses',
  u'friendly',
  u'beginners',
  u'enthusiasts',
  u'come',
  u'enjoy'],
 [u'ahusa',
  u'powell_library',
  u'exhibition',
  u'opening',
  u'reception',
  u'the',
  u'ucla',
  u'art',
  u'history',
  u'undergraduate',
  u'student',
  u'association',
  u'ahusa',
  u'organizing',
  u'exhibition',
  u'powell_library',
  u'rotunda',
  u'thursday',
  u'january',
  u'thursday',
  u'february',
  u'curatorial',
  u'statement',
  u'weather',
  u'ucla',
  u'los',
  u'angeles',
  u'perpetually',
  u'tempera

In [102]:
from collections import Counter
Counter(p.Y)

Counter({u'ART': 2, u'FOOD': 1, u'MUSIC': 1, u'WELLNESS': 1})